## Create a docker container and run

In [1]:
import docker

# Connect to Docker
client = docker.from_env()

# Function to run a new container
def run_jena_container():
    try:
        # Run the container with specific options
        container = client.containers.run(
            "stain/jena-fuseki",  # Image name
            name="jena",          # Container name
            detach=True,          # Run in detached mode (like -d)
            tty=True,             # Allocate a pseudo-TTY (like -it)
            ports={'3030/tcp': 3030},  # Port mapping (like -p 3030:3030)
            environment={"ADMIN_PASSWORD": "admin"},  # Environment variable
            stdin_open=True  # Keep STDIN open (for interactive mode)
        )
        print(f"Container '{container.name}' started successfully!")
    except docker.errors.APIError as e:
        print(f"Error starting the container: {e}")

# Run the Jena container
run_jena_container()


Error starting the container: 409 Client Error for http+docker://localhost/v1.47/containers/create?name=jena: Conflict ("Conflict. The container name "/jena" is already in use by container "f165546565a43a94e39b643f2d60b883d7a6228231d1fc6a9a8fcac6270a383d". You have to remove (or rename) that container to be able to reuse that name.")


## Run the existing container

In [8]:
import docker

# Connect to Docker
client = docker.from_env()

# Container name
container_name = "jena"

# Start the container
def start_container():
    try:
        container = client.containers.get(container_name)
        if container.status == "exited":
            container.start()
            print(f"Container '{container_name}' started successfully!")
        else:
            print(f"Container '{container_name}' is already running.")
    except docker.errors.NotFound:
        print(f"Container '{container_name}' not found.")


# Example usage:
start_container()  # Start the container


Container 'jena' started successfully!


## Stop the existing container

In [3]:
import docker

# Connect to Docker
client = docker.from_env()

# Container name
container_name = "jena"
# Stop the container
def stop_container():
    try:
        container = client.containers.get(container_name)
        if container.status == "running":
            container.stop()
            print(f"Container '{container_name}' stopped successfully!")
        else:
            print(f"Container '{container_name}' is not running.")
    except docker.errors.NotFound:
        print(f"Container '{container_name}' not found.")

# Example usage:
stop_container()   # Stop the container


Container 'jena' stopped successfully!


## Create a dataset named `ds`

In [26]:
import requests

# URL for the Fuseki Admin API to create a new dataset
fuseki_admin_url = "http://localhost:3030/$/datasets"

# Data for the new dataset
data = {
    'dbName': 'ds',  # Name of the new dataset
    'dbType': 'tdb2'  # Dataset type ('mem' for in-memory, 'tdb2' for persistent)
}


# Send the POST request to create the dataset with authentication
response = requests.post(fuseki_admin_url, data=data, auth=('admin', 'admin'))

# Check if the dataset was created successfully
if response.status_code == 200:
    print("Dataset created successfully!")
else:
    print(f"Failed to create dataset: {response.status_code} - {response.text}")


Failed to create dataset: 409 - Name already registered /ds



## Remove `ds`

In [27]:
# URL to delete the dataset 'ds'
delete_url = "http://localhost:3030/$/datasets/ds"

# Send the DELETE request to remove the dataset
response = requests.delete(delete_url, data=data, auth=('admin', 'admin'))

# Check if the dataset was deleted successfully
if response.status_code == 200:
    print("Dataset deleted successfully!")
else:
    print(f"Failed to delete dataset: {response.status_code} - {response.text}")


Dataset deleted successfully!


## Insert dummy data

In [20]:
from SPARQLWrapper import SPARQLWrapper, POST

# Set up the SPARQL Update endpoint for the 'foo' dataset
sparql = SPARQLWrapper("http://localhost:3030/ds/update")

# Set the credentials (assuming default username 'admin' and password 'your_password')
sparql.setCredentials("admin", "admin")

# Define a larger SPARQL INSERT query for RDF* data with more entities and metadata
insert_rdf_star = """
    INSERT DATA {
        # Relationships between people
        <http://example.org/Alice> <http://example.org/knows> <http://example.org/Bob> .
        << <http://example.org/Alice> <http://example.org/knows> <http://example.org/Bob> >> 
            <http://example.org/certainty> "high" ;
            <http://example.org/startedAt> "2021-01-01"^^<http://www.w3.org/2001/XMLSchema#date> ;
            <http://example.org/friendshipScore> "95"^^<http://www.w3.org/2001/XMLSchema#integer> .

        <http://example.org/Alice> <http://example.org/worksWith> <http://example.org/Charlie> .
        << <http://example.org/Alice> <http://example.org/worksWith> <http://example.org/Charlie> >> 
            <http://example.org/certainty> "medium" ;
            <http://example.org/startedAt> "2022-03-15"^^<http://www.w3.org/2001/XMLSchema#date> ;
            <http://example.org/workScore> "85"^^<http://www.w3.org/2001/XMLSchema#integer> .

        <http://example.org/Bob> <http://example.org/knows> <http://example.org/Charlie> .
        << <http://example.org/Bob> <http://example.org/knows> <http://example.org/Charlie> >> 
            <http://example.org/certainty> "low" ;
            <http://example.org/startedAt> "2020-06-20"^^<http://www.w3.org/2001/XMLSchema#date> ;
            <http://example.org/friendshipScore> "70"^^<http://www.w3.org/2001/XMLSchema#integer> .

        # Adding more metadata about the relationships
        <http://example.org/Charlie> <http://example.org/knows> <http://example.org/David> .
        << <http://example.org/Charlie> <http://example.org/knows> <http://example.org/David> >> 
            <http://example.org/certainty> "very high" ;
            <http://example.org/startedAt> "2019-09-05"^^<http://www.w3.org/2001/XMLSchema#date> ;
            <http://example.org/friendshipScore> "100"^^<http://www.w3.org/2001/XMLSchema#integer> .

        <http://example.org/David> <http://example.org/worksWith> <http://example.org/Alice> .
        << <http://example.org/David> <http://example.org/worksWith> <http://example.org/Alice> >> 
            <http://example.org/certainty> "high" ;
            <http://example.org/startedAt> "2018-11-11"^^<http://www.w3.org/2001/XMLSchema#date> ;
            <http://example.org/workScore> "90"^^<http://www.w3.org/2001/XMLSchema#integer> .

        # Additional metadata on people themselves
        <http://example.org/Alice> <http://example.org/age> "30"^^<http://www.w3.org/2001/XMLSchema#integer> .
        <http://example.org/Bob> <http://example.org/age> "35"^^<http://www.w3.org/2001/XMLSchema#integer> .
        <http://example.org/Charlie> <http://example.org/age> "28"^^<http://www.w3.org/2001/XMLSchema#integer> .
        <http://example.org/David> <http://example.org/age> "40"^^<http://www.w3.org/2001/XMLSchema#integer> .

        # Adding occupations
        <http://example.org/Alice> <http://example.org/occupation> "Engineer" .
        <http://example.org/Bob> <http://example.org/occupation> "Artist" .
        <http://example.org/Charlie> <http://example.org/occupation> "Data Scientist" .
        <http://example.org/David> <http://example.org/occupation> "Manager" .
    }
"""

# Set the query method to POST (for updates)
sparql.setQuery(insert_rdf_star)
sparql.setMethod(POST)

# Execute the insert query
sparql.query()
print("Larger RDF* dataset inserted successfully!")


Larger RDF* dataset inserted successfully!


## Retreive all the stored RDF-star data 

In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Set up the SPARQL Query endpoint for the 'ds' dataset
sparql = SPARQLWrapper("http://localhost:3030/ds/sparql")

# Define the SPARQL query to retrieve RDF* data
select_rdf_star = """
    SELECT ?s ?p ?o ?certainty WHERE {
      ?s ?p ?o .
      << ?s ?p ?o >> <http://example.org/certainty> ?certainty .
    }
"""

# Set the query method and return format
sparql.setQuery(select_rdf_star)
sparql.setReturnFormat(JSON)

# Execute the query and print results
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(f"Subject: {result['s']['value']}, Predicate: {result['p']['value']}, Object: {result['o']['value']}, Certainty: {result['certainty']['value']}")


Subject: http://example.org/Alice, Predicate: http://example.org/knows, Object: http://example.org/Bob, Certainty: high
Subject: http://example.org/David, Predicate: http://example.org/worksWith, Object: http://example.org/Alice, Certainty: high
Subject: http://example.org/Alice, Predicate: http://example.org/worksWith, Object: http://example.org/Charlie, Certainty: medium
Subject: http://example.org/Bob, Predicate: http://example.org/knows, Object: http://example.org/Charlie, Certainty: low
Subject: http://example.org/Charlie, Predicate: http://example.org/knows, Object: http://example.org/David, Certainty: very high


## Retreive all triples

In [25]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Set up the SPARQL Query endpoint for the 'foo' dataset
sparql = SPARQLWrapper("http://localhost:3030/ds/sparql")

# Define the SPARQL query to get all standard triples (no RDF-star)
select_standard_triples = """
    SELECT ?s ?p ?o WHERE {
      ?s ?p ?o .
    }
"""

# Set the query method and return format
sparql.setQuery(select_standard_triples)
sparql.setReturnFormat(JSON)

# Execute the query and print results
results = sparql.query().convert()

# Loop through the results and print each triple
for result in results["results"]["bindings"]:
    print(f"Subject: {result['s']['value']}, Predicate: {result['p']['value']}, Object: {result['o']['value']}")


Subject: http://example.org/Alice, Predicate: http://example.org/knows, Object: http://example.org/Bob
Subject: http://example.org/Alice, Predicate: http://example.org/worksWith, Object: http://example.org/Charlie
Subject: http://example.org/Alice, Predicate: http://example.org/age, Object: 30
Subject: http://example.org/Alice, Predicate: http://example.org/occupation, Object: Engineer
Subject: http://example.org/Bob, Predicate: http://example.org/knows, Object: http://example.org/Charlie
Subject: http://example.org/Bob, Predicate: http://example.org/age, Object: 35
Subject: http://example.org/Bob, Predicate: http://example.org/occupation, Object: Artist
Subject: {'subject': {'type': 'uri', 'value': 'http://example.org/Alice'}, 'predicate': {'type': 'uri', 'value': 'http://example.org/knows'}, 'object': {'type': 'uri', 'value': 'http://example.org/Bob'}}, Predicate: http://example.org/certainty, Object: high
Subject: {'subject': {'type': 'uri', 'value': 'http://example.org/Alice'}, 'pr

## Export the database as `example.ttl`

In [21]:
from SPARQLWrapper import SPARQLWrapper, TURTLE

# Set up the SPARQL Query endpoint for your dataset
sparql = SPARQLWrapper("http://localhost:3030/ds/sparql")

# Define the SPARQL CONSTRUCT query to get all triples
construct_query = """
    CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o . }
"""

# Set the query method and return format (Turtle)
sparql.setQuery(construct_query)
sparql.setReturnFormat(TURTLE)

# Execute the query and retrieve results in Turtle format
turtle_data = sparql.query().convert()

# Write the Turtle data to a file
with open('example.ttl', 'wb') as f:
    f.write(turtle_data)

print("Data exported as Turtle format.")


Data exported as Turtle format.


## Delete everything in the database. This does not delete the database

In [22]:
from SPARQLWrapper import SPARQLWrapper, JSON, POST

# Set up the SPARQL Query endpoint for the 'foo' dataset
sparql = SPARQLWrapper("http://localhost:3030/ds/update")

# Set the credentials (assuming default username 'admin' and password 'your_password')
sparql.setCredentials("admin", "admin")

# Define the SPARQL query to delete all triples
delete_all_triples = """
    DELETE WHERE {
      ?s ?p ?o .
    }
"""

# Set the query method and execute the update
sparql.setQuery(delete_all_triples)
sparql.setMethod(POST)  # Since it's an update, use POST
sparql.query()

print("All triples have been deleted from the database.")


All triples have been deleted from the database.


In [24]:
from SPARQLWrapper import SPARQLWrapper, POST
import requests

# Define the path to your Turtle file
file_path = "example.ttl"

# Set up the SPARQL Update endpoint for the 'foo' dataset
sparql_endpoint = "http://localhost:3030/ds/data"

# Set the headers for the request (important for content type Turtle)
headers = {
    'Content-Type': 'text/turtle'
}

# Read the Turtle file content
with open(file_path, 'r') as file:
    turtle_data = file.read()

# Perform the POST request to import the Turtle data
response = requests.post(sparql_endpoint, headers=headers, data=turtle_data, auth=('admin', 'admin'))

# Check if the import was successful
if response.status_code == 200:
    print("Turtle file imported successfully!")
else:
    print(f"Failed to import Turtle file. Status code: {response.status_code}, Response: {response.text}")


Turtle file imported successfully!
